In [2]:
import torch
from torch.utils.data import Dataset
from typing import Tuple

# typing 활용
# torch의 Dataset을 상속받아서 데이터셋 클래스 정의
# Dataset 은 구조체의 역할 정도만 기대하므로, 복잡한 기능은 넣지 않음
# 즉, 전처리 등의 기능은 넣을 필요가 없음. dataloader의 역할.
class CustomDataset(Dataset):
    def __init__(self, data: torch.Tensor, targets:torch.Tensor):
        self.data = data
        self.targets = targets
    
    def __len__(self) -> int:
        return len(self.data)
    
    def __getitem__(self, idx:int) -> Tuple[torch.Tensor, int]:
        #idx 에 해당하는 데이터와 레이블을 반환
        return self.data[idx], self.targets[idx]
    
# 예시
dummy_data = torch.randn(100, 28, 28) # 100개의 28x28 이미지
dummy_targets = torch.randint(0, 10, (100,))
my_dataset = CustomDataset(dummy_data, dummy_targets)
print(f"데이터셋 길이: {len(my_dataset)}")
print(f"첫 번째 데이터: {my_dataset[0][0].shape}, {my_dataset[0][1]}")

데이터셋 길이: 100
첫 번째 데이터: torch.Size([28, 28]), 3


In [6]:
from torch.utils.data import DataLoader

# DataLoader 는 Dataset 을 감싸서 사용함.
data_loader = DataLoader(
    dataset = my_dataset,
    batch_size = 16,
    shuffle = True,
    # num_workers = 2 # 워커가 안 됨 (윈도우 이슈
)

# for 루프를 통해 미니배치를 하나씩 꺼내 쓸 수 있음.
for images, labels in data_loader:
    print(f"배치 이미지 크기 : {images.shape}")
    print(f"배치 레이블 크기 : {labels.shape}")
    break # 첫 배치만 확인

배치 이미지 크기 : torch.Size([16, 28, 28])
배치 레이블 크기 : torch.Size([16])


In [7]:
# 학습 루프 내 사용 예시
optimizer = torch.optim.Adam(params=[], lr = 1e-3)
scaler = torch.amp.GradScaler()

for images, labels in data_loader:
    optimizer.zero_grad()

    with torch.amp.autocast() : # 해당 블록 안에서 AMP가 적용됨
        # 학습 부분에서만 amp.autocase(32비트를 16비트로 쪼개서 계산)를 사용하겠다!
        # 연산 최적화
        outputs = model(images)
        loss = loss_fn(outputs, labels)

    # loss 를 스케일링하고 backward()호출
    # 스케일링을 하는 이유 : 작은 값이 소실되는 것을 방지하기 위함.
    scaler.scale(loss).backward()
    # 파라미터 업데이트 (unscale 자동 수행)
    # 중요한 점은 optimiser 를 최적화 한 뒤에 스케일러를 업데이트 한다는 것.
    scaler.step(optimizer) 
    # 다음 반복을 위해 스케일러 업데이트
    scaler.update()

/Users/jonayeon/git/devops_hs/.venv/lib/python3.11/site-packages/torch/amp/grad_scaler.py:136: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


AttributeError: module 'torch' has no attribute 'optimizer'

In [ ]:
# Trainer 클래스와 콜백을 활용하여, MNIST 손글씨 숫자 데이터셋을 분류하는 MLP 모델 구축.
# 1. torchvision.dataset.MINIST를 활용하여 데이터셋과 데이터로더를 준비.
# 2. Trainer 객체 생성.
#    CheckpointCallback, EarlyStoppingCallback, LoggingCallback 사용
#    CheckpointCallback : 모델 체크포인트 저장
#    EarlyStoppingCallback : 검증 손실이 일정 epoch 동안 향상되지 않으면 학습 조기 종료
#    LoggingCallback : 학습 로그 출력 (epoch, loss)
# 3. 분류 97% 이상 달성. (lr, parmateter 조정)
# 4. 모든 랜덤 시드 (random, numpy, torch)를 고정했을 때, 항상 동일한 결과가 나와야 함. (재현성)
#    테스트 정확도 +-0.2%
# 5. 실행 후, 학습 로그가 정상적으로 출력되고 가장 좋은 성능의 모델의 체크포인프 파일(.pth)가 실제로 생성되어야 함.
